In [1]:
using Flux
using MLDatasets
using Statistics
using Random
using Optimisers
using LaplaceRedux
using Plots
using TaijaPlotting
include("laplace_utils.jl")
Random.seed!(42)

train_data = MLDatasets.MNIST(:train)
test_data = MLDatasets.MNIST(:test)

X_train = reshape(train_data.features, :, 60000)
y_train = Flux.onehotbatch(train_data.targets, 0:9)
X_test = reshape(test_data.features, :, 10000)
y_test = Flux.onehotbatch(test_data.targets, 0:9)


10×10000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  …  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  1     ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [3]:
X_train = Float32.(X_train)
X_test = Float32.(X_test)

println("X_train shape: ", size(X_train), ", type: ", eltype(X_train))
println("X_train min: ", minimum(X_train), ", max: ", maximum(X_train))
println("X_test shape: ", size(X_test), ", type: ", eltype(X_test))
println("X_test min: ", minimum(X_test), ", max: ", maximum(X_test))

X_train shape: (784, 60000), type: Float32
X_train min: 0.0, max: 1.0
X_test shape: (784, 10000), type: Float32
X_test min: 0.0, max: 1.0


In [16]:
model = Chain(
    Dense(784, 128, relu, init=Flux.glorot_uniform),
    Dense(128, 64, relu, init=Flux.glorot_uniform),
    Dense(64, 10, init=Flux.glorot_uniform)
)

loss(x, y) = Flux.crossentropy(softmax(model(x)), y)
opt = Optimisers.Adam(0.01)

state = Optimisers.setup(opt, model)

(layers = ((weight = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999))), σ = ()), (weight = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(0.01, (0.9, 0.999), 1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [17]:
data_train = [(X_train[:, i:i], y_train[:, i:i]) for i in 1:60000]
data_test = [(X_test[:, i:i], y_test[:, i:i]) for i in 1:10000]

10000-element Vector{Tuple{Matrix{Float32}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}}:
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 1; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [1; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 1; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 1;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 1;;])
 ⋮
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 1; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 1;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [1; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 1; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])
 ([0.0; 0.0; … ; 0.0; 0.0;;], [0; 0; … ; 0; 0;;])


In [18]:
num_epochs = 10
batch_size = 128
for epoch in 1:num_epochs
    perm = randperm(60000)
    X_train = X_train[:, perm]
    y_train = y_train[:, perm]
    data_train = data_train[perm]
    
    total_loss = 0.0
    num_batches = 0
    for i in 1:batch_size:60000
        x_batch = X_train[:, i:min(i+batch_size-1, 60000)]
        y_batch = y_train[:, i:min(i+batch_size-1, 60000)]
        
        (l, grads) = Flux.withgradient(model) do m
            Flux.crossentropy(softmax(m(x_batch)), y_batch)
        end
        
        state, model = Optimisers.update(state, model, grads[1])
        
        total_loss += l
        num_batches += 1
    end
    
    train_loss = total_loss / num_batches
    test_loss = loss(X_test, y_test)
    println("Epoch $epoch: Loss = $train_loss, test loss = $test_loss")
end

Epoch 1: Loss = 0.2311840876380899, test loss = 0.14064391
Epoch 2: Loss = 0.12437022879306696, test loss = 0.14867124
Epoch 3: Loss = 0.09940545714057203, test loss = 0.12151123
Epoch 4: Loss = 0.0909477540774386, test loss = 0.15791947
Epoch 5: Loss = 0.08758597794189446, test loss = 0.11913164
Epoch 6: Loss = 0.07573572247932508, test loss = 0.121748075
Epoch 7: Loss = 0.07286197372497732, test loss = 0.12765886
Epoch 8: Loss = 0.07262363233295743, test loss = 0.13884676
Epoch 9: Loss = 0.06302816210426827, test loss = 0.14173672
Epoch 10: Loss = 0.06396205308335795, test loss = 0.13160942


In [19]:
y_pred = softmax(model(X_test))
y_pred_labels = [argmax(y_pred[:, i]) - 1 for i in 1:10000]
y_true_labels = test_data.targets
accuracy = mean(y_pred_labels .== y_true_labels)
println("MAP ACC: $accuracy")


MAP ACC: 0.973


In [20]:
using CUDA

In [9]:
function apply_laplace(model, data_train, X_test, y_test, hessian_type, la_name)
    if CUDA.has_cuda()
        model = gpu(model)
        X_test = gpu(X_test)
        y_test = gpu(y_test)
        data_train = [(gpu(x), gpu(y)) for (x, y) in data_train]
    else
        println("CUDA Not Found, using CPU")
    end

    println("Apply $la_name (Hessian: $hessian_type)...")
    
    la = Laplace(model; likelihood=:classification, hessian_structure=hessian_type, subset_of_weights=:last_layer)
    fit!(la, data_train)
    optimize_prior!(la; verbosity=1, n_steps=100)
    
    y_pred_la = predict(la, X_test, ret_distr=true)

    y_pred_labels_la = [argmax(p.p) - 1 for p in y_pred_la]
    accuracy_la = mean(y_pred_labels_la .== vec(y_test))
    println("$la_name test set acc: $accuracy_la")

    _labels = 0:9 
    for target in _labels
        bernoulli_distributions = [LaplaceRedux.Bernoulli(p.p[target + 1]) for p in y_pred_la]
        y_onehot_test = Flux.unstack(Array(y_test)', 1)
        plt = Calibration_Plot(la, [y[target + 1] for y in y_onehot_test], bernoulli_distributions; n_bins=10)
        savefig(plt, "calibration_class_$(target)_$(la_name).png")
        println(" $target has a calibration curve: calibration_class_$(target)_$(la_name).png")
    end

    confidences = [maximum(p.p) for p in y_pred_la]
    mean_confidence = mean(confidences)
    println("AVG confidence: $mean_confidence")

    y_true = [argmax(Array(y_test)[:, i]) - 1 for i in 1:size(y_test, 2)]
    auroc_list = Float64[]
    for class_idx in 0:9
        y_true_bin = [yt == class_idx ? 1 : 0 for yt in y_true]
        y_score = [p.p[class_idx + 1] for p in y_pred_la]
        roc_obj = roc(y_score, y_true_bin)
        auc = auc(roc_obj)
        push!(auroc_list, auc)
        println("Class $class_idx with AUROC: $auc")
    end
    mean_auroc = mean(auroc_list)
    println("AUROC: $mean_auroc")
    
    println("$la_name DONE")
    return la
end

apply_laplace (generic function with 1 method)

In [21]:

la_kron = apply_laplace(model, data_train, X_test, y_test, :kron, "LA")


CUDA Not Found, using CPU
Apply LA (Hessian: kron)...


DimensionMismatch: DimensionMismatch: matrix is not square: dimensions are (10, 64)

In [10]:
la_full = apply_laplace(model, data_train, X_test, y_test, :full, "LA*")

CUDA Not Found, using CPU
Apply LA* (Hessian: full)...


┌ Info: Iteration 10: P₀=2.6899557264887024, σ=1.0
└ @ LaplaceRedux /Users/davidhuang/.julia/packages/LaplaceRedux/1bSKz/src/baselaplace/optimize_prior.jl:47


loss(exp.(logP₀), exp.(logσ)) = 4032.044273171899
Log likelihood: -3247.432263349771
Log det ratio: 1454.567556056358
Scatter: 114.6564635878981


┌ Info: Iteration 20: P₀=6.463462150737228, σ=1.0
└ @ LaplaceRedux /Users/davidhuang/.julia/packages/LaplaceRedux/1bSKz/src/baselaplace/optimize_prior.jl:47


loss(exp.(logP₀), exp.(logσ)) = 3962.522332576617
Log likelihood: -3247.432263349771
Log det ratio: 1154.6820306446407
Scatter: 275.4981078090516


...

┌ Info: Iteration 80: P₀=7.649904220863293, σ=1.0
└ @ LaplaceRedux /Users/davidhuang/.julia/packages/LaplaceRedux/1bSKz/src/baselaplace/optimize_prior.jl:47



Scatter: 326.06892229235496
loss(exp.(logP₀), exp.(logσ)) = 3960.6255285953903
Log likelihood: -3247.432263349771
Log det ratio: 1110.2522436085155
Scatter: 316.13428688272376


┌ Info: Iteration 90: P₀=7.4168277018912265, σ=1.0
└ @ LaplaceRedux /Users/davidhuang/.julia/packages/LaplaceRedux/1bSKz/src/baselaplace/optimize_prior.jl:47
┌ Info: Iteration 100: P₀=7.458982717172642, σ=1.0
└ @ LaplaceRedux /Users/davidhuang/.julia/packages/LaplaceRedux/1bSKz/src/baselaplace/optimize_prior.jl:47


loss(exp.(logP₀), exp.(logσ)) = 3960.617109552105
Log likelihood: -3247.432263349771
Log det ratio: 1108.4385934305537
Scatter: 317.93109897411466
LA* test set acc: 0.09961


UndefVarError: UndefVarError: `Calibration_Plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.